In [1]:
import math
import autograd.numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from matplotlib import animation
from autograd import grad

In [2]:
D = 2 # dimensionality of space (2D, 3D etc.)
N = 3 # number of bodies - currently Sun, Earth, Mars - we will increase this later

G = 6.67408e-11 # m^3 kg^{-1} s^{-1}
M_e = 5.972e24 # kg
R_e = 6.373e6 # m

v_e = math.sqrt(2*G*M_e/R_e) # m/s

print("Escape velocity: "+'{:.1f}'.format(v_e / 1e3)+ " km/s", ) # printf macro useful for formatted printing

Escape velocity: 11.2 km/s


In [3]:
# Masses in solar mass units
m_s = 1.0  
m_e = 0.000003 
m_m = 0.00000032

m = [m_s, m_e, m_m] # pack masses into a 1D array

M = np.repeat(m, D)  # repeat masses for each dimension
M

array([1.0e+00, 1.0e+00, 3.0e-06, 3.0e-06, 3.2e-07, 3.2e-07])

In [4]:
# Initial position vectors in X,Y coordinates
r_s = [0.0, 0.0]
r_e = [0.1, -1.0] # In astronomical units
r_m = [0.0, -1.524]; # In astronomical units

In [5]:
def potential_energy(R):
    r = np.reshape(R, (N, D)) # convert from 1d vector back to D-dimensional (here 2D)
    U = 0.0
    for i in range(N):
        for j in range(N):
            if i != j:
                r_ij = np.linalg.norm(r[i] - r[j]) # distance from i to j
                U += -G/2.0 * m[i]*m[j] / r_ij # contribution to potential
    return U

In [6]:
r0 = np.reshape([r_s, r_e, r_m], [N, D])
print(r0)
R0 = np.reshape(r0, N*D) # pack into 1D array
R0

[[ 0.     0.   ]
 [ 0.1   -1.   ]
 [ 0.    -1.524]]


array([ 0.   ,  0.   ,  0.1  , -1.   ,  0.   , -1.524])

In [7]:
grad_potential_energy = grad(potential_energy)
UR = grad_potential_energy(R0)
UR

array([-1.97256173e-17,  2.06451590e-16,  1.97256595e-17, -1.97255951e-16,
       -4.22051293e-23, -9.19563866e-18])

In [8]:
h = 1e-5
dU = np.zeros(N*D)
for i in range(N*D):
    Rp = R0.copy()
    Rp[i] += h
    dU[i] = (potential_energy(Rp) - potential_energy(R0)) / h
dU

array([-1.97246301e-17,  2.06449587e-16,  1.97266165e-17, -1.97257895e-16,
       -1.20350592e-23, -9.19569900e-18])

In [9]:
# Initial velocity vectors in X,Y coordinates
v_s = [0.0, 0.0] # In astronomical units per year
v_e = [-6.32, 0.0] # In astronomical units per year
v_m = [-5.05, 0.0] # In astronomical units per year

# pack into NxD array and vector log length N*D
v0 = np.reshape([v_s, v_e, v_m], [N, D])
print(v0)
V0 = np.reshape(v0, N*D) # pack into 1D array
V0

[[ 0.    0.  ]
 [-6.32  0.  ]
 [-5.05  0.  ]]


array([ 0.  ,  0.  , -6.32,  0.  , -5.05,  0.  ])

In [10]:
G = 37.95  # Gravitational constant

In [11]:
# Simulation time variables
dt = 0.005
t = np.arange(0.0, 2, dt)
T = len(t) # Total simulation points
T

400

In [12]:
def forward_euler(R0, V0, M, dt, T):

    n = len(R0)      # number of degrees of freedom (N x D)
    R = R0.copy()        # current position
    V = V0.copy()        # current velocity
    Rs = np.zeros([T, n])    # position history (trajectory) 
    Vs = np.zeros([T, n])    # velocity history

    for i in range(T):
        Rs[i] = R    # Store positions
        Vs[i] = V    # Store velocities

        V_i = V.copy()   # Store current values of velocities
        #print(potential_energy(R))
        A = -grad_potential_energy(R) / M # acceleration from Newton's 2nd law (F = ma)
        #print(A)
        V += A * dt     # Update velocities
        R += V_i * dt   # Update positions
    return (Rs, Vs)


R1, V1 = forward_euler(R0, V0, M, dt, T)

In [13]:
from ipywidgets import interactive

def plot_trajectory(traj, step=5):
    Rs = np.reshape(traj, [np.size(traj, 0), N, D])

    def animate(n=0):
        fig, ax = plt.subplots(figsize=(8, 8))
        plt.xlim(-2, 2)
        plt.ylim(-2, 2)   

        # Sun - yellow
        ax.scatter(Rs[n, 0, 0], Rs[n, 0, 1], c='y', s=30)
        ax.plot(Rs[:n, 0, 0], Rs[:n, 0, 1], 'y-')
                                
        # Earth - blue            
        ax.scatter(Rs[n, 1, 0], Rs[n, 1, 1], c='b', s=20)
        ax.plot(Rs[:n, 1, 0], Rs[:n, 1, 1], 'b-')
                                
        # Mars - red
        ax.scatter(Rs[n, 2, 0], Rs[n, 2, 1], c='r', s=10)
        ax.plot(Rs[:n, 2, 0], Rs[:n, 2, 1], 'r-')
                                
        # for the spaceship, see later...
        if np.size(Rs, 1) > 3:
            ax.scatter(Rs[n, 3, 0], Rs[n, 3, 1], c='g', s=5)
            ax.plot(Rs[:n, 3, 0], Rs[:n, 3, 1], c='g')

    return interactive(animate, n=(0, len(traj)-1, step))

In [14]:
plot_trajectory(R1)

interactive(children=(IntSlider(value=0, description='n', max=399, step=5), Output()), _dom_classes=('widget-i…

In [91]:
def symplectic_euler(R0, V0, M, dt, T):
    n = len(R0)          # number of degrees of freedom (N x D)
    R = R0.copy()        # current position
    V = V0.copy()        # current velocity
    Rs = np.zeros([T, n])  # position history (trajectory) 
    Vs = np.zeros([T, n])  # velocity history

    for i in range(T):
        Rs[i] = R     # Store positions
        Vs[i] = V     # Store velocities
        
        A = -grad_potential_energy(R) / M  # acceleration from Newton's 2nd law (F = ma)
        V += A * dt      # Update velocities
        R += V * dt      # Update positions
    
    return (Rs, Vs)

R2, V2 = symplectic_euler(R0, V0, M, dt, T);

In [92]:
plot_trajectory(R2)

interactive(children=(IntSlider(value=200, description=&#39;n&#39;, max=400, step=5), Output()), _dom_classes=(&#39;widget…

In [17]:
#unfinished
'''
N = 4 # change number of particles from 3 to 4
D = 2 # still modelling in 2D (x,y) plane

r0 = np.transpose([r_s, r_e, r_m])
r0_spc = np.transpose([0.101, -1.001]) # small displacement away from Earth    
r0p = np.concatenate(r0, r0_spc)
R0p = reshape(r0p, N*D)
print(R0p)

v0 = np.transpose([v_s, v_e, v_m])
v0_spc = np.transpose([6.7, 1.2])
v0p = np.concatenate(v0, v0_spc)
V0p = np.reshape(v0p, N*D)

V0p = np.concatenate(v0, v0_spc)
print(V0p)

m_spc = 1e-6

m = [m_s, m_e, m_m, m_spc] 
M = np.repeat(m, D)

(R, V) = symplectic_euler(R0p, V0p, M, dt, T);
'''


TypeError: only integer scalar arrays can be converted to a scalar index